In [5]:
import generate
import symae
import os
os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'

In [6]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
print("Num GPUs Available: ", len(gpus))

Num GPUs Available:  0


In [3]:
# Ok, What's in symae?
# Downsampler, DownsamplerDense, NuisanceEncoder, NuisanceEncoder1D, NuisanceEncoderDense1D, 
# SymmetricEncoder,SymmetricEncoder1D,SymmetricEncoderDense1D,DistributeZsym,LatentCat,DroppedLatentCat, 
# Mixer, Mixer1D, MixerDense1D, Upsampler, UpsamplerDense
# They are all Neural Nets. But where is the SymAE?

In [ ]:
encoder_input=tfk.Input(shape=(ntau,n1,n2,1), dtype='float32', name='encoder_input') #input shape => 10, 50, 410 = ntau,n1,n2 ###NONE
kernel_size=(5,5)
nfilt=64 #64
nz0=8 #symmetric encoder latent dimensions
nzi=22 #nuisance encoder latent dimensions. 


symencoder1=SymmetricEncoder(kernel_size,nfilt,[2,2,2,2],[2,2,2,2],nz0, [1,1,1]) #initialize model
xz0=symencoder1(encoder_input)
z0=tfkl.Flatten()(xz0)
nz0=z0.get_shape().as_list()[1]
symencoder=tfk.Model(encoder_input, z0, name='symencoder')
symencoder.summary()

nencoder1=NuisanceEncoder(kernel_size,nfilt,[2,2,2,2],[2,2,2,2],[1,1,1],nzi) #initialize model
xzi=nencoder1(encoder_input)
zi=tfkl.Flatten()(xzi)
nencoder=tfk.Model(encoder_input, zi, name='nencoder')
nencoder.summary()

decoder_input = tfk.Input(shape=(nzi*ntau+nz0), name='latentcode') #shape=(nzi*ntau+nz0) ###NONE

distzsym1=DistributeZsym(ntau, nz0, nzi) #initialize model
xdhat=distzsym1(decoder_input)
distzsym = tfk.Model(decoder_input, xdhat, name='distzsym')
distzsym.summary()


mixer_input = tfk.Input(shape=(ntau,nzi+nz0), name='mixer_input') #DOES FUSING OF COHERENT INFORMATION WITH EACH INSTANCE OF THE NUISANCE INFORMATION.just a placeholder for mixer1. shape=(ntau,nzi+nz0) ###NONE

mixer1=Mixer(kernel_size,nfilt,(10,10),n2,n1,[1,1,1]) #initialize model
xhat=mixer1(xdhat) #mixer_input #xdhat
mixer = tfk.Model(xdhat, xhat, name='mixer') #(mixer_input, xhat) #xdhat
mixer.summary()

znuisance=nencoder(encoder_input)
zsym=symencoder(encoder_input)
#xdist = distzsym(decoder_input)
#xpredict = mixer(xdhat)

#     report(upsampler,"upsampler")

